In [1]:
import torch
import numpy as np
import random
from IPython import display
from matplotlib import pyplot as plt
import torchvision
import torchvision.transforms as transforms
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  #防止jupyter崩溃

In [2]:
#下载MNIST手写数据集
mnist_train = torchvision.datasets.MNIST(root='./Datasets/MNIST', train=True,
download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.MNIST(root='./Datasets/MNIST', train=False,
download=True, transform=transforms.ToTensor())
#读取数据
batch_size = 32
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True,
num_workers=0)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False,
num_workers=0)

E:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
def dropout(X,drop_prob):
    X=X.float()
    assert 0 <= drop_prob <=1
    keep_prob = 1-drop_prob
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape)<keep_prob).float()
    return mask*X/keep_prob

In [ ]:
X=torch.arange(10).view(2,5)
print(dropout(X,0),'\n')
print(dropout(X,0.5),'\n')
print(dropout(X,1))

In [6]:
#初始化参数
num_inputs,num_outputs,num_hiddens1,num_hiddens2 = 784,10,256,256

W1 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens1,num_inputs)), dtype=torch.float32,requires_grad=True)
b1 = torch.zeros(num_hiddens1,requires_grad=True)
W2 = torch.tensor(np.random.normal(0, 0.01, size=(num_hiddens2,num_hiddens1)), dtype=torch.float32,requires_grad=True)
b2 = torch.zeros(num_hiddens2,requires_grad=True)
W3 = torch.tensor(np.random.normal(0, 0.01, size=(num_outputs,num_hiddens2)), dtype=torch.float32,requires_grad=True)
b3 = torch.zeros(num_outputs,requires_grad=True)
params =[W1,b1,W2,b2,W3,b3]

In [8]:
drop_prob1,drop_prob2=0.2,0.5

def net(X,is_training=True):
    X=X.view(-1,num_inputs)
    H1=(torch.matmul(X,W1.t())+b1).relu()
    if is_training:
        H1=dropout(H1,drop_prob1)
    H2=(torch.matmul(H1,W2.t())+b2).relu()
    if is_training:
        H2=dropout(H2,drop_prob2)
    return torch.matmul(H2,W3.t())+b3

In [9]:
def evaluate_accuracy(data_iter,net):
    acc_sum,n=0.0,0
    for X,y in data_iter:
        acc_sum+=(net(X,is_training=False).argmax(dim=1)==y).float().sum().item()
        n+=y.shape[0]
    return acc_sum/n

In [15]:
#定义随机梯度下降法
def SGD(paras,lr):
    for param in params:
        param.data -= lr * param.grad

In [18]:
#测试集loss
def evaluate_loss(data_iter,net):
    acc_sum,loss_sum,n = 0.0,0.0,0
    for X,y in data_iter:
        y_hat = net(X)
        acc_sum += (y_hat.argmax(dim=1)==y).sum().item()
        l = loss(y_hat,y) # l是有关小批量X和y的损失
        loss_sum += l.sum().item()*y.shape[0]
        n+=y.shape[0]
    return acc_sum/n,loss_sum/n

In [19]:
#定义模型训练函数
def train(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,optimizer=None):
    train_ls = []
    test_ls = []
    for epoch in range(num_epochs): # 训练模型一共需要num_epochs个迭代周期
        train_l_sum, train_acc_num,n = 0.0,0.0,0
        # 在每一个迭代周期中，会使用训练数据集中所有样本一次
        for X, y in train_iter: # x和y分别是小批量样本的特征和标签
            y_hat = net(X)
            l = loss(y_hat, y).sum() # l是有关小批量X和y的损失
            #梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward() # 小批量的损失对模型参数求梯度
            if optimizer is None:
                SGD(params,lr)
            else:
                optimizer.step()
            #计算每个epoch的loss
            train_l_sum += l.item()*y.shape[0]
            #计算训练样本的准确率
            train_acc_num += (y_hat.argmax(dim=1)==y).sum().item()
            #每一个epoch的所有样本数
            n+= y.shape[0]
        train_ls.append(train_l_sum/n)
        test_acc,test_l = evaluate_loss(test_iter,net)
        test_ls.append(test_l)
        print('epoch %d, train_loss %.6f,test_loss %f,train_acc %.6f,test_acc %.6f'%(epoch+1, train_ls[epoch],test_ls[epoch],train_acc_num/n,test_acc))
    return train_ls,test_ls

In [20]:
num_epochs,lr,batch_size=5,0.1,128
loss=torch.nn.CrossEntropyLoss()
train(net,train_iter,test_iter,loss,num_epochs,batch_size,params,lr,None)

epoch 1, train_loss 0.186851,test_loss 0.148770,train_acc 0.944317,test_acc 0.955300
epoch 2, train_loss 0.133903,test_loss 0.139640,train_acc 0.960050,test_acc 0.958000
epoch 3, train_loss 0.105571,test_loss 0.111442,train_acc 0.967933,test_acc 0.965400
epoch 4, train_loss 0.090396,test_loss 0.107504,train_acc 0.972850,test_acc 0.968100
epoch 5, train_loss 0.079016,test_loss 0.106666,train_acc 0.976083,test_acc 0.967500


([0.18685149675011634,
  0.13390267499809463,
  0.10557074166027208,
  0.09039612401618312,
  0.07901648317569246],
 [0.1487695613734424,
  0.13963997040428222,
  0.11144194802977145,
  0.10750422503910959,
  0.10666603716928512])